In [2]:
from langchain_community.document_loaders import PyPDFLoader

directory = "E:\\2024_GenAI\\LANGCHAIN\\ChatBot_with_Langchain_Pinecone\\data\\Canada.pdf"

def load_data(directory):
    loader = PyPDFLoader(directory)
    documents = loader.load()
    return documents

documents = load_data(directory)
len(documents)

65

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=500, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

511


In [7]:
print(docs[7].page_content)

• Total
  $2.472 trillion[5]
(16th)
• Per capita
  $60,495[5] (28th)
GDP (nominal) 2024 estimate
• Total
  $2.242 trillion[5]
(10th)
• Per capita
  $54,866[5] (18th)
Gini (2018)
  30.3[6]
medium
HDI (2022)
  0.935[7]
very high (18th)
Currency Canadian dollar ($)
(CAD)
Time zone UTC−3.5 to −8
• Summer (DST) UTC−2.5 to −7equality. It is one of the world's most ethnically diverse
and multicultural nations, the produc t of large-scale
immigration. Canada's long and complex relationship


In [15]:
# from langchain_community.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# client = AzureOpenAI(
#     api_version="2024-02-15-preview",
#     azure_deployment="atttestgpt35turbo",
#     azure_endpoint="https://cb-att-openai-instance.openai.azure.com/",
#     api_key=AZURE_OPENAI_KEY
# )

import os
from dotenv import load_dotenv
from langchain_community.embeddings import AzureOpenAIEmbeddings

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

embeddings = AzureOpenAIEmbeddings(
    deployment='vector-search-instance2',
    model='text-embedding-ada-002',
    openai_api_type='azure',
    azure_endpoint='https://cb-att-openai-instance.openai.azure.com/',
    api_key=AZURE_OPENAI_API_KEY,
)

In [17]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [46]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(
    api_key='7a0aca34-2555-4295-8507-3def5da9ca4f',
    environment='us-east-1'
)

index_name = 'langchain-chatbot'
index = pc.Index(index_name)
index
vectordb = PineconeVectorStore.from_documents(documents=docs, embedding=embeddings, index_name=index_name)

In [ ]:
def get_similar_docs(query, k=1, score=False):
    if score:
        similar_docs = vectordb.similarity_search_with_score(query, k=k)  
    else:
        similar_docs = vectordb.similarity_search(query, k=k)
    return similar_docs

query = "What is population in Canada?"
similar_docs = get_similar_docs(query)
similar_docs